In [1]:

import pandas as pd
from datetime import date
# # Run in case warnings should be ignored
import warnings
warnings.filterwarnings('ignore')

# info on website:
# --> "https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/"

# library for API-requests with API-key:
import requests

# basic access to URL, the key has to be requested before:
BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
key = "XXXXXXXXXXXXXXXXXXXXXXXXX"

# the form of the url to get the requests: 
'''https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/
    London,UK/2020-10-01/2020-12-31?key=YOUR_API_KEY'''  


'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/\n    London,UK/2020-10-01/2020-12-31?key=YOUR_API_KEY'

In [2]:

# function that makes a API request:
def create_api_data(city,start_date,end_date):
    
    # if the page is not found --> try/catch-block:
    try:
        
        # create url:
        url = f"{BASE_URL}/{city}/{start_date}/{end_date}"
        
        params = {"key":key,
                  # metric system in europe:
                  "unitGroup":"metric",
                  "contentType":"json"}
        
        resp = requests.get(url, params)
        
        return resp.json()
        
    except ValueError as e:
        
        # if it doesn't work, i know with the status_code what went wrong:
        print(resp.text,resp.status_code)
        
    finally:
        pass    


In [ ]:

# initialize a location (city):
chosen_city = "Innsbruck"

# calling the function to get the data set in json.format:
df01 = create_api_data(chosen_city,"2023-06-15","2023-06-29")


In [4]:

type(df01)


dict

In [6]:

df01.keys()


dict_keys(['queryCost', 'latitude', 'longitude', 'resolvedAddress', 'address', 'timezone', 'tzoffset', 'days', 'stations'])

In [7]:

# function for daily weather data with the infos i need:
def get_daily_data(df01):
    
    # check whether data can be processed:
    if not isinstance(df01, dict):
        print("Here went something wrong!")

    # create dataframe with existing data (days,temperature,etc.):
    dates = []
    temp = []
    tempmax = []
    tempmin = []
    humidity = []
    windspeed = []
    cloudcover = []
    sunrise = []
    sunset = []
    conditions = []
    pressure = []
    winddir = []

    # Iterate over the keys of the dictionary of data:
    for d in df01["days"]:
        dates.append(d["datetime"])
        temp.append(d["temp"])
        tempmax.append(d["tempmax"])
        tempmin.append(d["tempmin"])
        humidity.append(d["humidity"])
        windspeed.append(d["windspeed"])
        cloudcover.append(d["cloudcover"])
        sunrise.append(d["sunrise"])
        sunset.append(d["sunset"])
        conditions.append(d["conditions"])
        pressure.append(d["pressure"])
        winddir.append(d["winddir"])

    df_daily = pd.DataFrame(data = {f"Date_{df01['address']}" : dates,
                               "Temp": temp,                           
                               "Max_Temp" : tempmax,
                               "Min_Temp" : tempmin,
                               "Humidity" : humidity,
                               "Windspeed" : windspeed,
                               "Cloudcover" : cloudcover,
                               "Sunrise": sunrise,
                               "Sunset" : sunset,
                               "Conditions" : conditions,
                               "Pressure" : pressure,
                               "Winddir" : winddir})
    
    # date of today:
    date_today = date.today()

    # save the dataframe as csv-file:
    df_daily.to_csv(f"Daily_Weather_{chosen_city}{date_today}.csv",index=False)
    
    return df_daily.info()


In [8]:

# calling the function to get the daily data set with the requested data:
get_daily_data(df01)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date_Innsbruck  15 non-null     object 
 1   Temp            15 non-null     float64
 2   Max_Temp        15 non-null     float64
 3   Min_Temp        15 non-null     float64
 4   Humidity        15 non-null     float64
 5   Windspeed       15 non-null     float64
 6   Cloudcover      15 non-null     float64
 7   Sunrise         15 non-null     object 
 8   Sunset          15 non-null     object 
 9   Conditions      15 non-null     object 
 10  Pressure        15 non-null     float64
 11  Winddir         15 non-null     float64
dtypes: float64(8), object(4)
memory usage: 1.5+ KB


In [13]:

# load the new saved data set:
date_today = date.today()
df_daily = pd.read_csv(f"Daily_Weather_{chosen_city}{date_today}.csv")
df_daily


,Date_Innsbruck,Temp,Max_Temp,Min_Temp,Humidity,Windspeed,Cloudcover,Sunrise,Sunset,Conditions,Pressure,Winddir
0,2023-06-15,17.0,24.1,10.1,50.2,24.1,51.7,05:17:44,21:12:17,Partially cloudy,1016.0,48.9
1,2023-06-16,15.1,19.2,11.5,67.7,15.9,75.4,05:17:42,21:12:43,Partially cloudy,1018.2,32.4
2,2023-06-17,16.6,25.1,10.2,61.4,13.1,51.7,05:17:43,21:13:07,Partially cloudy,1017.1,29.8
3,2023-06-18,19.4,30.1,9.5,50.8,17.6,24.2,05:17:46,21:13:28,Partially cloudy,1015.3,1.9
4,2023-06-19,22.1,29.2,16.9,56.3,23.9,61.7,05:17:51,21:13:47,Partially cloudy,1014.5,37.5
5,2023-06-20,23.4,30.5,15.5,60.6,28.3,43.3,05:18:00,21:14:03,Partially cloudy,1014.7,155.4
6,2023-06-21,22.7,29.8,17.5,67.5,23.8,51.1,05:18:10,21:14:17,"Rain, Partially cloudy",1017.4,349.9
7,2023-06-22,24.5,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
8,2023-06-23,18.9,24.7,15.0,73.3,21.1,82.5,05:18:39,21:14:37,"Rain, Partially cloudy",1020.5,71.6
9,2023-06-24,16.9,21.6,13.0,69.0,16.8,75.8,05:18:57,21:14:44,"Rain, Partially cloudy",1025.2,57.0


In [10]:

# function for hourly weather data with the infos i need:
def get_hourly_data(df01):
    
    # check whether data can be processed:
    if not isinstance(df01, dict):
        print("Here went something wrong!")

    # create dataframe with existing data (days,temperature,etc.):
    dates = []
    hour = []
    temp = []
    tempmax = []
    tempmin = []
    humidity = []
    windspeed = []
    cloudcover = []
    sunrise = []
    sunset = []
    conditions = []
    pressure = []
    winddir = []    

    for d in df01["days"]:
            for h in d["hours"]:
                dates.append(d["datetime"])
                hour.append(f"{h['datetime']}")
                temp.append(h["temp"])
                tempmax.append(d["tempmax"])
                tempmin.append(d["tempmin"])
                humidity.append(d["humidity"])
                windspeed.append(d["windspeed"])
                cloudcover.append(d["cloudcover"])
                sunrise.append(d["sunrise"])
                sunset.append(d["sunset"])
                conditions.append(d["conditions"])
                pressure.append(d["pressure"])
                winddir.append(d["winddir"])

    df_hourly = pd.DataFrame(data = {f"Date_{df01['address']}" : dates,
                               "Hour": hour,
                               "Temp": temp,                           
                               "Max_Temp" : tempmax,
                               "Min_Temp" : tempmin,
                               "Humidity" : humidity,
                               "Windspeed" : windspeed,
                               "Cloudcover" : cloudcover,
                               "Sunrise": sunrise,
                               "Sunset" : sunset,
                               "Conditions" : conditions,
                               "Pressure" : pressure,
                               "Winddir" : winddir})
    
    # date of today:
    date_today = date.today()

    # save the dataframe as csv-file:
    df_hourly.to_csv(f"Hourly_Weather_{chosen_city}{date_today}.csv",index=False)
    
    return df_hourly.info()


In [11]:

# calling the function to get the hourly data set with the requested data:
get_hourly_data(df01)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date_Innsbruck  360 non-null    object 
 1   Hour            360 non-null    object 
 2   Temp            360 non-null    float64
 3   Max_Temp        360 non-null    float64
 4   Min_Temp        360 non-null    float64
 5   Humidity        360 non-null    float64
 6   Windspeed       360 non-null    float64
 7   Cloudcover      360 non-null    float64
 8   Sunrise         360 non-null    object 
 9   Sunset          360 non-null    object 
 10  Conditions      360 non-null    object 
 11  Pressure        360 non-null    float64
 12  Winddir         360 non-null    float64
dtypes: float64(8), object(5)
memory usage: 36.7+ KB


In [12]:

# load the new saved data set:
date_today = date.today()
df_hourly = pd.read_csv(f"Hourly_Weather_{chosen_city}{date_today}.csv")
# hourly data set for a single day, f.e. "2023-06-22":
df_hourly[df_hourly.Date_Innsbruck == "2023-06-22"]


,Date_Innsbruck,Hour,Temp,Max_Temp,Min_Temp,Humidity,Windspeed,Cloudcover,Sunrise,Sunset,Conditions,Pressure,Winddir
168,2023-06-22,00:00:00,18.2,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
169,2023-06-22,01:00:00,18.3,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
170,2023-06-22,02:00:00,18.5,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
171,2023-06-22,03:00:00,17.6,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
172,2023-06-22,04:00:00,17.4,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
173,2023-06-22,05:00:00,17.3,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
174,2023-06-22,06:00:00,17.3,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
175,2023-06-22,07:00:00,17.6,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
176,2023-06-22,08:00:00,18.6,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1
177,2023-06-22,09:00:00,20.3,33.2,17.3,61.1,23.1,54.2,05:18:23,21:14:29,Partially cloudy,1015.7,241.1


In [ ]:

# all the keys and values of the requested data:
df01


{'queryCost': 360,
 'latitude': 47.268,
 'longitude': 11.3913,
 'resolvedAddress': 'Innsbruck, Tirol, Österreich',
 'address': 'Innsbruck',
 'timezone': 'Europe/Vienna',
 'tzoffset': 2.0,
 'days': [{'datetime': '2023-06-15',
   'datetimeEpoch': 1686780000,
   'tempmax': 24.1,
   'tempmin': 10.1,
   'temp': 17.0,
   'feelslikemax': 24.1,
   'feelslikemin': 10.1,
   'feelslike': 17.0,
   'dew': 5.9,
   'humidity': 50.2,
   'precip': 0.0,
   'precipprob': 0.0,
   'precipcover': 0.0,
   'preciptype': None,
   'snow': 0.0,
   'snowdepth': 0.0,
   'windgust': 39.6,
   'windspeed': 24.1,
   'winddir': 48.9,
   'pressure': 1016.0,
   'cloudcover': 51.7,
   'visibility': 10.0,
   'solarradiation': 325.7,
   'solarenergy': 26.9,
   'uvindex': 10.0,
   'sunrise': '05:17:44',
   'sunriseEpoch': 1686799064,
   'sunset': '21:12:17',
   'sunsetEpoch': 1686856337,
   'moonphase': 0.9,
   'conditions': 'Partially cloudy',
   'description': 'Partly cloudy throughout the day.',
   'icon': 'partly-cloudy-